In [1]:
import pandas as pd
df_ = pd.read_csv("Desktop/titanic.csv")
df = df_.copy()
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1.0,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,0.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,0.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,0.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


# Veri Temizleme

In [2]:
df.isnull().sum()

pclass          1
survived        1
name            1
sex             1
age           264
sibsp           1
parch           1
ticket          1
fare            2
cabin        1015
embarked        3
boat          824
body         1189
home.dest     565
dtype: int64

In [3]:
# Kullanılmayacak sütunları veri setinden çıkarıyorum.
df.drop(['body','home.dest'], axis=1, inplace=True)

In [4]:
# Yaş sütununu eksik değerlerini median ile dolduruyorum
df['age'] = df['age'].fillna(df['age'].median())

In [5]:
# Cabin'in ilk harfini alalım (ön işlem)
df['Cabin_Code'] = df['cabin'].apply(lambda x: x[0] if pd.notnull(x) else None)

# Fare eksik olan her satır için, aynı Cabin_Code'a sahip kişilerin ortalama ücretini kullan
def fill_fare(row):
    if pd.isnull(row['fare']) and pd.notnull(row['Cabin_Code']):
        # Aynı kabin koduna sahip kişilerin ortalama fare'ını al
        similar_cabin_group = df[(df['Cabin_Code'] == row['Cabin_Code']) & (df['fare'].notnull())]
        if not similar_cabin_group.empty:
            return similar_cabin_group['fare'].mean()
    return row['fare']

# Uygulama
df['fare'] = df.apply(fill_fare, axis=1)

In [6]:
# Ücret sütununu eksik değerlerini median ile dolduruyorum
df['fare'] = df['fare'].fillna(df['fare'].median())

In [7]:
# Varış yerleri sütununu eksik değerlerini mod ile dolduruyorum.
df['embarked'] = df['embarked'].fillna(df['embarked'].mode()[0])

In [8]:
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,Cabin_Code
0,1.0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,B
1,1.0,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,C
2,1.0,0.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,C
3,1.0,0.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,C
4,1.0,0.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,C


In [13]:
# Cabin NaN olanlara odaklandım
missing_cabin_df = df[df['cabin'].isnull()].copy()

In [14]:
# Cabin değeri olanlara odaklanalım
available_cabin_df = df[df['cabin'].notnull()].copy()

In [15]:
# FamilySize sütunun oluşturuyorum
df['FamilySize'] = df['sibsp'] + df['parch'] +1
# +1 kendimide dahil etmek için ekledim
print(df['FamilySize'].value_counts())

FamilySize
1.0     790
2.0     235
3.0     159
4.0      43
6.0      25
5.0      22
7.0      16
11.0     11
8.0       8
Name: count, dtype: int64


In [16]:
# Cabin'i tahmin etmek için fare ve FamilySize eşleşmesi kullanalım
for idx, row in missing_cabin_df.iterrows():
    # Aynı fare ve FamilySize'e sahip olan ve cabin bilgisi olanları bul
    matches = available_cabin_df[
        (available_cabin_df['fare'] == row['fare']) &
        (available_cabin_df['FamilySize'] == row['FamilySize'])
    ]
    
    if not matches.empty:
        # İlk eşleşen kişinin cabin'ini kullan
        df.at[idx, 'cabin'] = matches.iloc[0]['cabin']

In [17]:
df['FareRounded'] = df['fare'].round(0)

In [18]:
df.isnull().sum()

pclass            1
survived          1
name              1
sex               1
age               0
sibsp             1
parch             1
ticket            1
fare              0
cabin           729
embarked          0
boat            824
Cabin_Code     1015
FamilySize        1
FareRounded       0
dtype: int64

In [19]:
# Cabin'i tahmin etmek için fare ve pclass eşleşmesi kullanalım
for idx, row in missing_cabin_df.iterrows():
    # Aynı fare ve pclass'e sahip olan ve cabin bilgisi olanları bul
    matches = available_cabin_df[
        (available_cabin_df['fare'] == row['fare']) &
    (available_cabin_df['pclass'] == row['pclass'])
    ]
    
    if not matches.empty:
        # İlk eşleşen kişinin cabin'ini kullan
        df.at[idx, 'cabin'] = matches.iloc[0]['cabin']

In [20]:
df['cabin'] = df['cabin'].fillna('U')

In [21]:
df.isnull().sum()

pclass            1
survived          1
name              1
sex               1
age               0
sibsp             1
parch             1
ticket            1
fare              0
cabin             0
embarked          0
boat            824
Cabin_Code     1015
FamilySize        1
FareRounded       0
dtype: int64

In [22]:
# boat değeri olanlara odaklanalım
available_boat_df = df[df['boat'].notnull()].copy()

In [23]:
# boat NaN olanlara odaklandım
missing_boat_df = df[df['boat'].isnull()].copy()

In [24]:
# Cabin'i tahmin etmek için fare ve FamilySize eşleşmesi kullanalım
for idx, row in missing_boat_df.iterrows():
    # Aynı fare ve FamilySize'e sahip olan ve cabin bilgisi olanları bul
    matches = available_boat_df[
        (available_boat_df['fare'] == row['fare']) &
    (available_boat_df['ticket'] == row['ticket']) &
    (available_boat_df['embarked'] == row['embarked']) &
        (available_boat_df['FamilySize'] == row['FamilySize'])
    ]
    
    if not matches.empty:
        # İlk eşleşen kişinin cabin'ini kullan
        df.at[idx, 'boat'] = matches.iloc[0]['boat']

In [25]:
df.isnull().sum()

pclass            1
survived          1
name              1
sex               1
age               0
sibsp             1
parch             1
ticket            1
fare              0
cabin             0
embarked          0
boat            735
Cabin_Code     1015
FamilySize        1
FareRounded       0
dtype: int64

In [26]:
df.head(10)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,Cabin_Code,FamilySize,FareRounded
0,1.0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,B,1.0,211.0
1,1.0,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,C,4.0,152.0
2,1.0,0.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,11,C,4.0,152.0
3,1.0,0.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,11,C,4.0,152.0
4,1.0,0.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,11,C,4.0,152.0
5,1.0,1.0,"Anderson, Mr. Harry",male,48.0000,0.0,0.0,19952,26.5500,E12,S,3,E,1.0,27.0
6,1.0,1.0,"Andrews, Miss. Kornelia Theodosia",female,63.0000,1.0,0.0,13502,77.9583,D7,S,10,D,2.0,78.0
7,1.0,0.0,"Andrews, Mr. Thomas Jr",male,39.0000,0.0,0.0,112050,0.0000,A36,S,NaN,A,1.0,0.0
8,1.0,1.0,"Appleton, Mrs. Edward Dale (Charlotte Lamson)",female,53.0000,2.0,0.0,11769,51.4792,C101,S,D,C,3.0,51.0
9,1.0,0.0,"Artagaveytia, Mr. Ramon",male,71.0000,0.0,0.0,PC 17609,49.5042,C90,C,NaN,None,1.0,50.0


In [27]:
def clean_boat(val):
    if pd.isnull(val):
        return '2'  # NaN olanları 2 yap
    try:
        # Eğer sadece sayıysa (örneğin '5'), dönüşüm başarılı olur
        int(val)
        return val  # olduğu gibi bırak
    except:
        # Sayıya çevrilemiyorsa, içinde karakter veya boşluk vardır
        return '2'

# Uygula
df['boat'] = df['boat'].apply(clean_boat)

In [28]:
df.isnull().sum()

pclass            1
survived          1
name              1
sex               1
age               0
sibsp             1
parch             1
ticket            1
fare              0
cabin             0
embarked          0
boat              0
Cabin_Code     1015
FamilySize        1
FareRounded       0
dtype: int64

In [29]:
# Burada da Cabin_Code kısmını siliyorum.
df.drop(['Cabin_Code'], axis=1, inplace=True)

In [30]:
df.tail()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,FamilySize,FareRounded
1305,3.0,0.0,"Zabour, Miss. Thamine",female,28.0,1.0,0.0,2665,14.4542,U,C,2,2.0,14.0
1306,3.0,0.0,"Zakarian, Mr. Mapriededer",male,26.5,0.0,0.0,2656,7.2250,U,C,2,1.0,7.0
1307,3.0,0.0,"Zakarian, Mr. Ortin",male,27.0,0.0,0.0,2670,7.2250,U,C,2,1.0,7.0
1308,3.0,0.0,"Zimmerman, Mr. Leo",male,29.0,0.0,0.0,315082,7.8750,U,S,2,1.0,8.0
1309,NaN,NaN,NaN,NaN,28.0,NaN,NaN,NaN,14.4542,U,S,2,NaN,14.0


In [31]:
df.isnull().sum()

pclass         1
survived       1
name           1
sex            1
age            0
sibsp          1
parch          1
ticket         1
fare           0
cabin          0
embarked       0
boat           0
FamilySize     1
FareRounded    0
dtype: int64

# Feature Engineering aşamasına geçiyorum. Bu kısımda ham verilerden daha anlamlı ve model için faydalı olabilecek yeni değişkenler oluşturacam

In [20]:
# Title ünvan değişikliği oluşturdum

In [32]:
# Name kolonundan ünvanları (Mr, Mrs, Miss, vs.) çıkaralım
df['Title'] = df['name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
df['Title'].value_counts()

Title
Mr          757
Miss        260
Mrs         197
Master       61
Rev           8
Dr            8
Col           4
Mlle          2
Ms            2
Major         2
Capt          1
Sir           1
Dona          1
Jonkheer      1
Countess      1
Don           1
Mme           1
Lady          1
Name: count, dtype: int64

In [52]:
# Ünvanı guruplandırıdım
df['Title'] = df['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 
                                   'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
df['Title'] = df['Title'].replace('Mlle', 'Miss')
df['Title'] = df['Title'].replace('Ms', 'Miss')
df['Title'] = df['Title'].replace('Mme', 'Mrs')

In [33]:
# Ünvanları sayısal değerlere dönüştürdüm
df['Title'] = df['Title'].map({'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3, 'Rare': 4})
df['Title'] = df['Title'].fillna(4)

In [34]:
# Yaş kategorisini oluşturma yapıyorum
def age_category(age):
    if age <=18:
        return 0 # Çocuklar
    elif age <= 30:
        return 1 # Genç
    elif age <= 50:
        return 2 # Yetişkin
    else:
        return 3 # Yaşlı
df['AgeCategory'] = df['age'].apply(age_category)
df['AgeCategory'].value_counts()

AgeCategory
1    680
2    342
0    193
3     95
Name: count, dtype: int64

In [35]:
fare_bins = [0, 7.91, 14.454, 31, df['fare'].max()]
fare_labels = [0, 1, 2, 3]
df['FareCategory'] = pd.cut(df['fare'], bins=fare_bins, labels=fare_labels)
df['FareCategory'].value_counts()

FareCategory
2    333
3    330
0    320
1    310
Name: count, dtype: int64

In [36]:
#İsimleri Title çıkartma yapiyorum
import re

def get_title(name):
    if isinstance(name, str):  # name string ise işle
        title_search = re.search(r' ([A-Za-z]+)\.', name)  # raw string kullandık
        if title_search:
            return title_search.group(1)
    return "Unknown"  # NaN veya string değilse Unknown döndür

In [37]:
df['Title'] = df['name'].apply(get_title)
print(df['Title'].value_counts())

Title
Mr          757
Miss        260
Mrs         197
Master       61
Dr            8
Rev           8
Col           4
Ms            2
Major         2
Mlle          2
Jonkheer      1
Don           1
Countess      1
Lady          1
Dona          1
Sir           1
Capt          1
Mme           1
Unknown       1
Name: count, dtype: int64


In [39]:
# Yaygın olmayan unvanları Other yapalım
common_titles = ['Mr', 'Mrs', 'Miss', 'Master']
df['Title'] = df['Title'].apply(lambda x: x if x in common_titles else 'Other')

print(df['Title'].value_counts())

Title
Mr        757
Miss      260
Mrs       197
Master     61
Other      35
Name: count, dtype: int64


In [40]:
# Label encoding
title_mapping = {'Mr':0, 'Mrs':1, 'Miss':2, 'Master':3, 'Other':4}
df['Title'] = df['Title'].map(title_mapping)

In [41]:
print(df['Title'].value_counts())

Title
0    757
2    260
1    197
3     61
4     35
Name: count, dtype: int64


In [42]:
# Cinsiyet değişkjeninin sayısal değerlere çevirdim
df['sex'] = df['sex'].map({'male':0, 'female':1})

In [43]:
df['Embarked'] = df['embarked'].map({'S': 0, 'C': 1, 'Q': 2})

In [44]:
print(df['Embarked'].value_counts())

Embarked
0    917
1    270
2    123
Name: count, dtype: int64


In [45]:
# FamilySize sütunun oluşturuyorum
df['FamilySize'] = df['sibsp'] + df['parch'] +1
# +1 kendimide dahil etmek için ekledim
print(df['FamilySize'].value_counts())

FamilySize
1.0     790
2.0     235
3.0     159
4.0      43
6.0      25
5.0      22
7.0      16
11.0     11
8.0       8
Name: count, dtype: int64


In [46]:
# Aile göre gruplandırma
def family_size_group(size):
    if size == 1:
        return 0
    elif 2 <= size <= 4:
        return 1
    else:
        return 2

df['FamilySizeGroup'] = df['FamilySize'].apply(family_size_group)
print(df['FamilySizeGroup'].value_counts())

FamilySizeGroup
0    790
1    437
2     83
Name: count, dtype: int64


In [47]:
df['boat'].value_counts()

boat
2     851
15     46
10     40
4      40
14     40
13     39
11     32
8      31
3      29
5      29
16     28
9      26
6      26
7      24
12     24
1       5
Name: count, dtype: int64

In [48]:
def categorize_boat(val):
    try:
        val_int = int(val)
        if 1 <= val_int <= 5:
            return 0
        elif 6 <= val_int <= 10:
            return 1
        elif 11 <= val_int <= 16:
            return 2
        else:
            return np.nan  # veya örneğin 3 diyebilirsin
    except:
        return np.nan  # sayıya çevrilemeyenler

# Yeni kategorik sütunu oluşturuyorum
df['BoatCategory'] = df['boat'].apply(categorize_boat)
df['BoatCategory'].value_counts()

BoatCategory
0    954
2    209
1    147
Name: count, dtype: int64

In [49]:
df['cabin'].value_counts()

cabin
U      681
D       60
E10     60
F38     55
F2      43
      ... 
B11      1
D45      1
D49      1
B94      1
D6       1
Name: count, Length: 187, dtype: int64

In [50]:
def extract_cabin_category(val):
    if pd.isnull(val):
        return np.nan  # veya return "Unknown"
    else:
        return str(val)[0]  # ilk harf alınır

# Yeni sütun oluştur
df['Cabin_Category'] = df['cabin'].apply(extract_cabin_category)
df['Cabin_Category'].value_counts()

Cabin_Category
U    681
F    173
E    122
C    105
D    105
B     75
A     43
G      5
T      1
Name: count, dtype: int64

In [51]:
df['FareCategory'].value_counts()

FareCategory
2    333
3    330
0    320
1    310
Name: count, dtype: int64

In [52]:
# Cabin_Category değerleri için geçerli olanlar
valid_cabins = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T', 'U']

# Yeni sütun oluştur
df['Cabin_Categorys'] = df.apply(
    lambda row: row['FareCategory'] if row['Cabin_Category'] in valid_cabins else np.nan,
    axis=1
)
df['Cabin_Categorys'].value_counts()

Cabin_Categorys
2.0    333
3.0    330
0.0    320
1.0    310
Name: count, dtype: int64

# Model eğitim kısmım

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd

# 1. Kullandığım sütunlar
features = ['pclass', 'sex', 'AgeCategory', 'FareCategory', 'Embarked', 'Title', 'FamilySizeGroup', 'BoatCategory', 'Cabin_Categorys']

# Kategorik değişkenler için one-hot encoding
df_model = pd.get_dummies(df[features], drop_first=True)

# Hedef değişken
y = df['survived']

# 2. Veri setini ayırma
X_train, X_test, y_train, y_test = train_test_split(df_model, y, test_size=0.2, random_state=42)

# Eksik verileri temizleme
X_train = X_train.dropna(subset=['pclass', 'sex'])
y_train = y_train[X_train.index]  # X_train ile aynı indeksleri koru

# 3. Decision Tree modelini oluşturdum ve eğittim
# Parametreleri ayarlama kısmı
model = DecisionTreeClassifier(
    max_depth=5,  # Ağacın maksimum derinliği (aşırı öğrenmeyi önlemek için)
    min_samples_split=10,  # Bölünme için minimum örnek sayısı
    min_samples_leaf=5,    # Yaprak düğümler için minimum örnek sayısı
    random_state=42
)

model.fit(X_train, y_train)

# Tahmin ve değerlendirme kısmı
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.851145038167939
Classification Report:
               precision    recall  f1-score   support

         0.0       0.81      0.96      0.88       150
         1.0       0.93      0.71      0.80       112

    accuracy                           0.85       262
   macro avg       0.87      0.83      0.84       262
weighted avg       0.86      0.85      0.85       262

Confusion Matrix:
 [[144   6]
 [ 33  79]]


In [61]:
# Test verisini oluşturuyorum
test_data = pd.DataFrame([
    # 1. Sınıf Kadınlar için hayatta kalma senaryoları.
    {'pclass': 0, 'sex': 1, 'AgeCategory': 0, 'FareCategory': 1, 'Embarked':0, 'Title': 0, 'FamilySizeGroup': 3, 'BoatCategory': 0, 'Cabin_Categorys': 0},
    {'pclass': 0, 'sex': 1, 'AgeCategory': 0, 'FareCategory': 2, 'Embarked':0, 'Title': 1, 'FamilySizeGroup': 3, 'BoatCategory': 0, 'Cabin_Categorys': 1},
    {'pclass': 1, 'sex': 1, 'AgeCategory': 1, 'FareCategory': 0, 'Embarked':0, 'Title': 1, 'FamilySizeGroup': 1, 'BoatCategory': 1, 'Cabin_Categorys': 2},
    {'pclass': 1, 'sex': 1, 'AgeCategory': 1, 'FareCategory': 2, 'Embarked':0, 'Title': 0, 'FamilySizeGroup': 0, 'BoatCategory': 1, 'Cabin_Categorys': 0},
    {'pclass': 2, 'sex': 1, 'AgeCategory': 2, 'FareCategory': 2, 'Embarked':0, 'Title': 0, 'FamilySizeGroup': 0, 'BoatCategory': 2, 'Cabin_Categorys': 1},
    {'pclass': 2, 'sex': 1, 'AgeCategory': 2, 'FareCategory': 1, 'Embarked':0, 'Title': 1, 'FamilySizeGroup': 0, 'BoatCategory': 0, 'Cabin_Categorys': 0},
    {'pclass': 3, 'sex': 1, 'AgeCategory': 3, 'FareCategory': 0, 'Embarked':0, 'Title': 0, 'FamilySizeGroup': 0, 'BoatCategory': 3, 'Cabin_Categorys': 1},
    {'pclass': 3, 'sex': 1, 'AgeCategory': 3, 'FareCategory': 2, 'Embarked':0, 'Title': 1, 'FamilySizeGroup': 0, 'BoatCategory': 0, 'Cabin_Categorys': 0}
])

# One-hot encoding uyguladım.
test_data_encoded = pd.get_dummies(test_data, drop_first=True)

# Eğitim verisinde olan ama test verisinde olmayan sütunları ekledim
missing_cols = set(X_train.columns) - set(test_data_encoded.columns)
for col in missing_cols:
    test_data_encoded[col] = 0

# Sütunları eğitim verisiyle aynı sıraya getirdim
test_data_encoded = test_data_encoded[X_train.columns]

# Tahminleri yapıyorum.
predictions = model.predict(test_data_encoded)

# Sonuçları gösteriyorum.
for i, result in enumerate(predictions, 1):
    print(f"{i}. Yolcu → {'Hayatta kaldı ✅' if result == 1 else 'Hayatta kalamadı ❌'}")

1. Yolcu → Hayatta kaldı ✅
2. Yolcu → Hayatta kaldı ✅
3. Yolcu → Hayatta kaldı ✅
4. Yolcu → Hayatta kaldı ✅
5. Yolcu → Hayatta kaldı ✅
6. Yolcu → Hayatta kalamadı ❌
7. Yolcu → Hayatta kaldı ✅
8. Yolcu → Hayatta kalamadı ❌
